In [1]:

import pandas as pd

files_path="/Users/vanshaj/Work/GitHub/Quant_Labs/Application/Data/Assets Data/EQUITY/INDIA"
symbol ="/Users/vanshaj/Work/GitHub/Quant_Labs/Application/Data/India50_stocks.parquet"


ds = pd.read_parquet(symbol)["Symbol"]

for i in ds:
     df = pd.read_parquet(f"{files_path}/{i}_history.parquet")



In [ ]:
import numpy as np
import pandas as pd
from scipy.linalg import svd
